**ARTICLE 1 - ANGELE MENDY**

**TEST TECHNIQUE WEB DATA ANALYST MENTORAT**

In [ ]:
!pip install -q pandas


In [ ]:
import pandas as pd


In [ ]:
from google.colab import files

# Uploadez votre fichier CSV
uploaded = files.upload()


In [ ]:
import pandas as pd

extract = list(uploaded.keys())[0]

# Chargement du fichier CSV dans un DataFrame
extract = pd.read_csv(extract)

# Affichage des premières lignes du DataFrame
extract.head()


In [ ]:
# Copie du DataFrame
df = extract.copy()


In [ ]:
df.describe()


In [ ]:
df.isnull().sum()


In [ ]:
# Affichage des noms des colonnes totalement vides
columns_with_all_nulls = df.columns[df.isnull().all()]
print("Colonnes totalement vides :", columns_with_all_nulls)

# Suppression des colonnes totalement vides
df = df.drop(columns=columns_with_all_nulls, axis=1)




In [ ]:
# Affichage des noms des colonnes totalement vides
columns_with_all_nulls = df.columns[df.isnull().all()]
print("Colonnes totalement vides :", columns_with_all_nulls)

# Suppression des colonnes totalement vides
df = df.drop(columns=columns_with_all_nulls, axis=1)

# Affichage du DataFrame après la suppression
print(df)


In [ ]:
# La liste des colonnes
print(df.info)

In [ ]:
# Liste des colonnes de dates
date_columns = ['b_date_proposition', 'b_date_creation', 'b_date_maj_statut', 'j_date_inscription']

# Conversion des colonnes de dates
for col in date_columns:
    df[col] = pd.to_datetime(df[col], errors='coerce')

# Types de données après la conversion
print(df.dtypes)
print(df.head())


In [ ]:
# Liste des colonnes de dates
date_columns = ['b_date_proposition', 'b_date_creation', 'b_date_maj_statut', 'j_date_inscription']

# Boucle pour traiter chaque colonne de date
for col in date_columns:
    # La colonne est déjà convertie en datetime?
    df[col] = pd.to_datetime(df[col], errors='coerce')

    # Création une nouvelle colonne pour stocker la date
    df[f'{col}_date'] = df[col].dt.date

    # Création une nouvelle colonne pour stocker l'heure
    df[f'{col}_heure'] = df[col].dt.time

    # Supp de l'heure de la colonne d'origine
    df[col] = df[col].dt.floor('d')

# Premières lignes du DataFrame avec les nouvelles colonnes
df[date_columns + [f'{col}_date' for col in date_columns] + [f'{col}_heure' for col in date_columns]].head()


In [ ]:
# Calcul de la colonne "délai de matching"
df['delai_matching'] = (df['b_date_proposition']-df['b_date_creation']).dt.days

# Affichage de lignes aléatoires
print(df[['b_date_proposition', 'b_date_creation', 'delai_matching']].sample(frac=1))
# Affichage des valeurs uniques et leur nombre pour la colonne 'delai_matching2'
unique_values_counts = df['delai_matching'].value_counts()

# Affichage des valeurs uniques
print("Valeurs uniques :")
print(unique_values_counts.index)

# Résultat
print("\nNombre d'occurrences :")
print(unique_values_counts)

In [ ]:
# Calcul de la colonne "délai de matching"
df['delai_matching2'] = (df['b_date_creation'] - df['b_date_proposition']).dt.days

# Affichage du résultat
print(df[['b_date_proposition', 'b_date_creation', 'delai_matching2']])

# Affichage des valeurs uniques et leur nombre pour la colonne 'delai_matching2'
unique_values_counts2 = df['delai_matching2'].value_counts()

# Affichage des valeurs uniques
print("Valeurs uniques :")
print(unique_values_counts2.index)

# Résultat
print("\nNombre d'occurrences :")
print(unique_values_counts2)


In [ ]:
# Affichage des valeurs uniques et leur nombre pour la colonne 'delai_matching'
unique_values_counts = df['delai_matching'].value_counts()

# Affichage des valeurs uniques
print("Valeurs uniques :")
print(unique_values_counts.index)

# Résultat
print("\nNombre d'occurrences :")
print(unique_values_counts)


Question? Les dates 'b_date_creation' et 'b_date_proposition' ont-elle ete inversees?


In [ ]:
# Filtrer les lignes où la date de création est antérieure à la date de proposition
inconsistent_rows = df[df['b_date_creation'] < df['b_date_proposition']]

# Afficher les lignes complètes avec les colonnes spécifiées
print('''inconsistent_rows = toutes les lignes où la date de création est
      antérieure à la date de proposition.
      Ici, on trouve une incohérence dans vos données.''')
print('')
print(inconsistent_rows[['b_id', 'b_date_proposition', 'b_date_creation']])


**Correction de l'anomalie détectée dans les données.**

In [ ]:
# Afficher les lignes complètes des données incohérentes
print(df.loc[inconsistent_rows.index, ['b_date_proposition_date', 'b_date_creation_date']])


ICI je decide d'interverir les donnees des colonnes ['b_date_proposition_date', 'b_date_creation_date']

In [ ]:
df.sample(10)


In [ ]:
#Verification (il sagit des 16 #-2)
df[df['b_date_creation'] < df['b_date_proposition']]

In [ ]:
# Inversion des noms de colonnes
df = df.rename(columns={'b_date_proposition': 'temp_col', 'b_date_creation': 'b_date_proposition', 'temp_col': 'b_date_creation'})


In [ ]:
df[df['b_date_creation'] < df['b_date_proposition']]


In [ ]:
df.sample(10)

In [ ]:

# Calcul de la colonne "délai de matching"
df['delai_matching'] = (df['b_date_creation'] - df['b_date_proposition']).dt.days

# Calcul du délai moyen
delai_moyen = df['delai_matching'].mean()

# Affichage du résultat
print(f"Délai moyen entre la proposition et la création : {delai_moyen} jours")


In [ ]:
# Suppression de la colonne "delai_matching"
df = df.drop(columns=['delai_matching'])


**1.Requête SQL permet de calculer le délai moyen de matching**


SELECT AVG(DATEDIFF(b_date_creation, b_date_proposition)) AS delai_moyen_matching
FROM df_cleaned


**2.Code Python pour évaluer quel est le critère le plus important dans la réussite d'un matching.**

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Calcul la corrélation de Pearson entre b_score et chaque colonne numérique
correlations = df.corr()['b_score'].sort_values(ascending=False)

# Affichage des résultats
print(correlations)







1. **Variables positivement corrélées avec 'b_score':**
   - `j_admin_associe_id` (Corrélation = 0.379323) : Les bénévoles ayant un administrateur associé ont tendance à avoir des scores plus élevés.
   - `b_delai_acceptation_minutes` (Corrélation = 0.270684) : Un délai d'acceptation plus long est associé à des scores plus élevés.

2. **Variables négativement corrélées avec 'b_score':**
   - `delai_matching` (Corrélation = -0.270554) : Un délai de matching plus long est associé à des scores plus bas.
   - `j_bam_fait` (Corrélation = -0.427676) : Les bénévoles qui ont déjà interagi avec BAM (Bénévoles d'Avenir) ont tendance à avoir des scores plus bas.

3. **Variables avec une corrélation intéressante:**
   - `j_besoin_postuler_present` (Corrélation = -0.113970) : La présence du besoin de postuler est légèrement négativement corrélée avec les scores. Cela peut indiquer que les bénévoles ayant moins de besoins de postuler ont tendance à avoir des scores plus élevés.

4. **Variables à considérer:**
   - `b_duree_mois` (Corrélation = -0.195480) et `b_duree_binome` (Corrélation = -0.196732) : La durée du binôme et la durée en mois sont négativement corrélées avec les scores. Cela suggère que des durées plus longues peuvent être associées à des scores plus bas.

5. **Variables sans corrélation significative:**
   - `j_consentement_newsletter`, `j_ase`, etc. : Ces variables ont des corrélations proches de zéro, indiquant une faible association linéaire avec 'b_score'.



In [ ]:

# Colonnes d'intérêt
selected_columns = [
    'b_score', 'b_date_proposition', 'b_delai_acceptation_minutes', 'b_duree_mois', 'b_duree_binome',
    'delai_matching', 'j_admin_associe_id', 'j_besoin_decouvrir_present', 'j_nb_besoins', 'j_est_boursier',
    'j_besoin_postuler_present', 'b_id', 'j_bam_fait', 'j_appel_validation_fait'
]

# Sélection des données correspondantes
selected_data = df[selected_columns]

# Calcul de la corrélation de Pearson
correlations = selected_data.corr()

# Heatmap
plt.figure(figsize=(12, 10))
sns.heatmap(correlations, cmap='coolwarm', annot=True, fmt=".2f")
plt.title("Corrélation entre les colonnes sélectionnées")
plt.show()


In [ ]:
# Enregistrer la corrélation dans un fichier CSV
correlations.to_csv('correlations.csv', index=True)



1. **`b_score` avec d'autres variables** :
   - **`b_score`** présente une corrélation positive significative avec `j_admin_associe_id` (0.379323), ce qui suggère qu'une augmentation de **`b_score`** est associée à une augmentation de `j_admin_associe_id`.
   - La corrélation avec `j_est_boursier` est négative (-0.152784), indiquant une association inverse entre le score et le statut boursier.

2. **`b_delai_acceptation_minutes` avec d'autres variables** :
   - La corrélation avec `b_duree_mois` et `b_duree_binome` est faible, indiquant une faible association avec la durée du binôme.
   - La corrélation avec `delai_matching` est également faible (0.014515), suggérant une faible association avec le délai de matching.

3. **`j_admin_associe_id` avec d'autres variables** :
   - La corrélation avec `j_besoin_decouvrir_present` est positive (0.245488), indiquant une relation positive.
   - La corrélation avec `j_nb_besoins` est aussi positive (0.377570), suggérant une association plus forte.

4. **`j_besoin_postuler_present` avec d'autres variables** :
   - La corrélation avec `j_bam_fait` est négative (-0.246770), suggérant une relation inverse entre le besoin de postuler et la réalisation de l'appel à la BAM.
   - La corrélation avec `j_nb_besoins` est positive (0.091688), indiquant une association plus faible.

5. **`b_id` avec d'autres variables** :
   - La corrélation avec `j_appel_validation_fait` est élevée (0.561833), suggérant une forte association entre l'ID et la réalisation de l'appel à la validation.


In [ ]:


# Sélectionner toutes les colonnes de votre DataFrame
all_columns = df.columns

# Calculer la matrice de corrélation
correlation_matrix = df[all_columns].corr()

# Créer un masque pour la moitié supérieure de la matrice
mask = np.triu(np.ones_like(correlation_matrix, dtype=bool))

# Définir la taille de la figure
plt.figure(figsize=(16, 12))

# Créer le heatmap avec Seaborn
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f", mask=mask)

# Afficher le plot
plt.show()


In [ ]:
# Enregistrement la matrice de corrélation dans un fichier CSV
correlation_matrix.to_csv('correlation_matrix.csv', index=True)




1. **`j_besoin_anglais_present` avec d'autres variables** :
   - La corrélation avec `j_memo_lu` est de 0.109284, indiquant une faible association positive.
   - La corrélation avec `j_video_vue` est de 0.160614, suggérant une association légèrement plus forte.
   - La corrélation avec `j_mono_besoin` est de -0.160614, indiquant une association négative, probablement liée à des besoins monomatière.

2. **`j_memo_lu` avec d'autres variables** :
   - La corrélation avec `j_video_vue` est de -0.205826, indiquant une association négative.
   - La corrélation avec `j_mono_besoin` est de -0.160614, ce qui est cohérent avec la précédente corrélation, indiquant une association négative avec les besoins monomatériaux.

3. **`j_video_vue` avec d'autres variables** :
   - La corrélation avec `j_age` est de 0.160614, indiquant une association positive légère.
   - La corrélation avec `j_etudiant_etranger` est de 0.451895, suggérant une association plus forte avec les étudiants étrangers.

4. **`j_mono_besoin` avec d'autres variables** :
   - La corrélation avec `j_age` est de -0.160614, indiquant une association négative légère.
   - La corrélation avec `j_besoin_performer_present` est de 0.080048, indiquant une association positive.

5. **`j_binome_regional_souhaite` avec d'autres variables** :
   - Il y a des valeurs NaN, indiquant qu'il n'y a pas suffisamment de données pour calculer la corrélation avec d'autres variables.

6. **`j_age` avec d'autres variables** :
   - La corrélation avec `j_appel_validation_fait` est de 0.492418, indiquant une association positive avec la réalisation de l'appel à la validation.
   - La corrélation avec `j_ase` est de -0.009819, indiquant une association très faible avec le statut ASE.

7. **`delai_matching` avec d'autres variables** :
   - La corrélation avec `b_id` est de 0.633110, suggérant une forte association positive entre le délai de matching et l'ID.


**CONCLUSION**

1. **Délai d'acceptation :**
   - Les données ont révélé que des délais d'acceptation courts sont associés à des scores plus élevés, suggérant un impact positif sur le succès du matching.

2. **Intérêt pour la découverte des besoins :**
   - Les données ont montré une corrélation positive entre la présence de l'intérêt ("j_besoin_decouvrir_present") et "b_score", confirmant que cet intérêt est associé à des scores plus élevés.

3. **Durée du binôme :**
   - Les données ont indiqué des corrélations négatives entre les colonnes liées à la durée du binôme, telles que "b_duree_binome" et "b_duree_mois", et "b_score", suggérant que des durées plus longues sont associées à des scores plus élevés. Cela souligne l'importance des binômes prolongés dans la construction de relations bénévole-jeune solides.

4. **`j_appel_validation_fait` (Corrélation avec `delai_matching`: 0.492418)** :
   - Une corrélation positive significative est observée avec le fait d'avoir réalisé l'appel à la validation (`j_appel_validation_fait`). Cela suggère que les jeunes ayant effectué cet appel ont tendance à avoir des délais de matching plus courts. Cela pourrait indiquer que l'engagement actif des jeunes dans le processus contribue à une meilleure réussite du matching.

5. **`j_etudiant_etranger` (Corrélation avec `delai_matching`: -0.076326)** :
   - Bien que cette corrélation soit moins forte, une corrélation négative est observée avec le statut d'étudiant étranger (`j_etudiant_etranger`). Cela indique que les étudiants étrangers ont tendance à avoir des délais de matching plus courts. Cela pourrait être lié à une priorité accrue accordée aux étudiants étrangers dans le processus de matching.



Angele MENDY, le 05/12/2023